# Cryptocurrency Market Analysis and Email Notification System

## Introduction
This project is designed to fetch real-time cryptocurrency market data, generate investment reports, and send automated email notifications containing insights. The system utilizes the CoinGecko API to extract market trends and Python libraries for data processing, scheduling, and email automation.

## Project Overview
The script performs the following tasks:
- Fetches cryptocurrency market data from CoinGecko API.
- Analyzes and filters the top and bottom 10 cryptocurrencies based on 24-hour price change.
- Generates a CSV report containing key market statistics.
- Sends an automated email with the market summary and attached CSV report.
- Uses scheduling to run the report generation and email dispatch daily at a fixed time.

## Components of the System

### 1. API Request and Data Extraction
- The script requests market data from CoinGecko API.
- Retrieves key cryptocurrency metrics such as price, market cap, all-time highs/lows, and price changes.
- Converts the fetched JSON data into a structured Pandas DataFrame.

### 2. Data Processing and Report Generation
- Extracts the top 10 and bottom 10 cryptocurrencies based on 24-hour price change percentage.
- Stores the processed data into a CSV file, including key metrics.

### 3. Email Notification System
- Configures an SMTP server for sending emails.
- Formats an email containing market insights and attaches the generated CSV report.
- Handles exceptions to ensure the reliability of the email dispatch function.

### 4. Automation with Scheduling
- Uses the `schedule` library to run the script daily at 8:00 AM.
- Ensures timely execution of market data retrieval and email dispatch.

## Steps in the Process

### Step 1: Fetching Cryptocurrency Data
- Calls the CoinGecko API to retrieve real-time market data.
- Parses the response and structures it into a DataFrame.

### Step 2: Data Cleaning and Analysis
- Selects relevant columns such as `id`, `current_price`, `market_cap`, `price_change_percentage_24h`, etc.
- Identifies the top 10 best-performing and worst-performing cryptocurrencies.
- Adds the current date for reference.

### Step 3: Saving Data to CSV
- Generates two reports: one for top 10 cryptocurrencies and one for bottom 10 cryptocurrencies.
- Saves the reports as CSV files for further analysis and record-keeping.

### Step 4: Sending Email Notifications
- Configures SMTP settings to send an email from a Gmail account.
- Constructs an email containing market highlights.
- Attaches the CSV report to the email.
- Handles potential exceptions during email transmission.

### Step 5: Automating Execution with Schedule
- Uses the `schedule` module to execute the `get_crypto_report()` function daily at 8:00 AM.
- Ensures consistency in data reporting and email updates.


In [ ]:
import pandas as pd
import requests
import smtplib
import schedule
import time
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
import email.encoders

# API URL and parameters
API_URL = 'https://api.coingecko.com/api/v3/coins/markets'
PARAMS = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 250,
    'page': 1
}

# Email credentials
SENDER_EMAIL = "your_email@gmail.com"
RECEIVER_EMAIL = "receiver_email@gmail.com"
PASSWORD = "your_password"


def send_mail(subject, body, filename):
    """Sends an email with the given subject, body, and attachment."""
    server = "smtp.gmail.com"
    port = 587
    
    message = MIMEMultipart()
    message['From'] = SENDER_EMAIL
    message['To'] = RECEIVER_EMAIL
    message['Subject'] = subject
    
    # Attach email body
    message.attach(MIMEText(body, 'plain'))
    
    # Attach the file
    with open(filename, 'rb') as file:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(file.read())
        email.encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{filename}"')
        message.attach(part)
    
    # Send the email
    try:
        with smtplib.SMTP(host=server, port=port) as server:
            server.starttls()
            server.login(SENDER_EMAIL, PASSWORD)
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")


def get_crypto_report():
    """Fetches cryptocurrency data, processes it, and sends an email report."""
    response = requests.get(API_URL, params=PARAMS)
    
    if response.status_code == 200:
        print("Successfully fetched data.")
        data = response.json()
        df = pd.DataFrame(data)
        
        # Select relevant columns
        df = df[['id', 'current_price', 'market_cap', 'price_change_percentage_24h',
                 'high_24h', 'low_24h', 'ath', 'atl']]
        
        today = datetime.today().strftime('%d-%m-%Y')
        df['date'] = today
        
        # Get top and bottom 10 cryptocurrencies
        top_10 = df.nlargest(10, 'price_change_percentage_24h')
        bottom_10 = df.nsmallest(10, 'price_change_percentage_24h')
        
        filename = f'top_10_{today}.csv'
        df.to_csv(filename, index=False)
        
        print("Data processing complete.")
        
        # Prepare email content
        subject = "Top 10 Cryptocurrencies to Invest In"
        body = f"""
        Good Morning!
        Hope you are doing well.
        
        Here are the best recommendations for {today}:
        
        Top 10 cryptocurrencies based on 24H price change:
        {top_10.to_string(index=False)}
        
        Bottom 10 cryptocurrencies based on 24H price change:
        {bottom_10.to_string(index=False)}
        
        Regards,
        Crypto Report Bot
        """
        
        send_mail(subject, body, filename)
    else:
        print("Failed to fetch data.")

# Schedule the task daily at 8:00 AM
schedule.every().day.at("08:00").do(get_crypto_report)

if __name__ == '__main__':
    while True:
        schedule.run_pending()
        time.sleep(60)


# Conclusion
This project provides an automated solution for tracking cryptocurrency market trends, making it easier for investors to stay informed. By integrating API requests, data analysis, email automation, and scheduling, the system ensures timely delivery of valuable market insights.

## Future Enhancements
- Implement data visualization for better insights.
- Add support for multiple currencies.
- Enhance email formatting with HTML reports.
- Store historical data for trend analysis.
